In [37]:
import numpy as np 
import pandas as pd 
import cv2
import cvzone
from ultralytics import YOLO
import math
from tracker import*
import time

In [2]:
model = YOLO('yolov8s.pt')

In [3]:
def RGB(event, x, y, flags, param):
    if event == cv2.EVENT_MOUSEMOVE :  
        point = [x, y]
        print(point)

In [4]:
cv2.namedWindow('RGB')
video_path = 'input_video.mp4'
cap = cv2.VideoCapture(video_path)
cv2.waitKey(1)
cv2.destroyAllWindows()

In [5]:
my_file = open("classes.txt", "r")
data = my_file.read()
class_list = data.split("\n") 

In [6]:
tracker = Tracker()
detected_ids = set()
crossed_ids = set()
cy1 = 194
cy2 = 220

In [39]:
count=0
min_height_threshold = 100
min_width_threshold = 50
line_start = (543, 94)
line_end = (756, 410)

frame_rate = 5  # Desired frame rate is 5 fps
frame_interval = 1 / frame_rate  # Time interval between frames
start_time = time.time()
save = cv2.VideoWriter('output_video.mp4', cv2.VideoWriter_fourcc(*'MJPG'),5, (1020,500))
while True:
    ret, frame = cap.read()

    if not ret:
        break
    current_time = time.time()

    if current_time - start_time >= frame_interval:
        start_time = current_time
        frame = cv2.resize(frame, (1020, 500))

        results = model.predict(frame)
        a = results[0].boxes.data
        px = pd.DataFrame(a).astype("float")

        list = []
        
        #getting the coordinates of the bounding box 
        for index, row in px.iterrows():
            x1 = int(row[0])
            y1 = int(row[1])
            x2 = int(row[2])
            y2 = int(row[3])
            d = int(row[5])

            c = class_list[d]
            if 'person' in c:
                list.append([x1,y1,x2,y2])

        bbox_id = tracker.update(list)
        for bbox in bbox_id:
            x3, y3, x4, y4, id = bbox
            cx = int((x3 + x4) / 2)
            cy = int((y3 + y4) / 2)
            height = y4 - y3
            width=x4-x3
            if height > min_height_threshold and width > min_width_threshold:
                if cx > line_end[0]:
                    #drawing the box and tracking circle
                    cv2.circle(frame, (cx, cy), 4, (255, 0, 255), -1)
                    cv2.rectangle(frame, (x3, y3), (x4, y4), (255, 0, 0), 2)
                    distance = abs((line_end[1] - line_start[1]) * cx -
                                   (line_end[0] - line_start[0]) * cy +
                                   line_end[0] * line_start[1] - line_end[1] * line_start[0]) / \
                               math.sqrt((line_end[1] - line_start[1]) ** 2 + (line_end[0] - line_start[0]) ** 2)

                    # Calculate the radius of the circle
                    radius = math.sqrt((line_end[1] - line_start[1]) ** 2 + (line_end[0] - line_start[0]) ** 2) / 2

                    # Check if the circle intersects with the line
                    if distance <= radius:
                        if id in detected_ids:
                            continue
                        else:
                            count += 1
                            detected_ids.add(id)
        #The area 
        cv2.line(frame, line_start, (1000, 5), (0, 255, 0), 2)
        cv2.line(frame, line_end, (1020, 380), (0, 255, 0), 2)
        cv2.line(frame, line_start, line_end, (0, 255, 0), 2)
        cv2.putText(frame, f'The Area', (650, 253), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        cv2.putText(frame, f'People Entred The Area: {count}', (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

        save.write(frame)
        cv2.imshow("RGB", frame)
    if cv2.waitKey(10) & 0xFF == 27:
        break

cap.release()
save.release()
cv2.destroyAllWindows()


0: 320x640 1 person, 1 bicycle, 167.5ms
Speed: 1.0ms preprocess, 167.5ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person, 1 bicycle, 196.5ms
Speed: 4.0ms preprocess, 196.5ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person, 1 bicycle, 176.5ms
Speed: 2.0ms preprocess, 176.5ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 2 persons, 1 bicycle, 196.0ms
Speed: 2.0ms preprocess, 196.0ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 2 persons, 176.5ms
Speed: 2.0ms preprocess, 176.5ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person, 163.6ms
Speed: 2.0ms preprocess, 163.6ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person, 1 bicycle, 186.5ms
Speed: 6.0ms preprocess, 186.5ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person, 181.5ms
Speed: 4.0ms prep


0: 320x640 1 person, 1 bicycle, 166.6ms
Speed: 2.0ms preprocess, 166.6ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person, 1 bicycle, 172.5ms
Speed: 1.0ms preprocess, 172.5ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person, 1 bicycle, 177.5ms
Speed: 3.0ms preprocess, 177.5ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person, 1 bicycle, 171.5ms
Speed: 2.0ms preprocess, 171.5ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person, 171.5ms
Speed: 2.0ms preprocess, 171.5ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person, 168.5ms
Speed: 2.0ms preprocess, 168.5ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person, 1 bicycle, 173.5ms
Speed: 2.0ms preprocess, 173.5ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person, 1 bicycle, 174.5ms
Speed: 2


0: 320x640 1 person, 2 bicycles, 264.3ms
Speed: 4.0ms preprocess, 264.3ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person, 2 bicycles, 266.3ms
Speed: 4.0ms preprocess, 266.3ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person, 2 bicycles, 259.3ms
Speed: 4.0ms preprocess, 259.3ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person, 2 bicycles, 196.5ms
Speed: 4.0ms preprocess, 196.5ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person, 2 bicycles, 169.6ms
Speed: 3.0ms preprocess, 169.6ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person, 2 bicycles, 168.5ms
Speed: 2.0ms preprocess, 168.5ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person, 2 bicycles, 168.5ms
Speed: 2.0ms preprocess, 168.5ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person


0: 320x640 1 person, 1 bicycle, 1 car, 259.3ms
Speed: 3.0ms preprocess, 259.3ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person, 1 bicycle, 1 car, 264.3ms
Speed: 4.0ms preprocess, 264.3ms inference, 3.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person, 1 bicycle, 1 car, 255.3ms
Speed: 3.0ms preprocess, 255.3ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person, 1 bicycle, 264.3ms
Speed: 4.0ms preprocess, 264.3ms inference, 3.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 bicycle, 1 car, 266.3ms
Speed: 4.0ms preprocess, 266.3ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person, 1 bicycle, 1 car, 266.3ms
Speed: 4.0ms preprocess, 266.3ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person, 1 bicycle, 1 car, 256.9ms
Speed: 4.0ms preprocess, 256.9ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 


0: 320x640 5 persons, 1 bicycle, 1 tennis racket, 237.4ms
Speed: 4.0ms preprocess, 237.4ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 3 persons, 1 bicycle, 173.5ms
Speed: 2.0ms preprocess, 173.5ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 3 persons, 1 bicycle, 171.0ms
Speed: 2.0ms preprocess, 171.0ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 4 persons, 1 bicycle, 175.5ms
Speed: 3.0ms preprocess, 175.5ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 5 persons, 1 bicycle, 169.5ms
Speed: 2.0ms preprocess, 169.5ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 5 persons, 1 bicycle, 178.5ms
Speed: 3.0ms preprocess, 178.5ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 6 persons, 1 bicycle, 188.5ms
Speed: 2.0ms preprocess, 188.5ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0:

...


0: 320x640 3 persons, 1 bench, 249.3ms
Speed: 3.0ms preprocess, 249.3ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 3 persons, 256.3ms
Speed: 4.0ms preprocess, 256.3ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 3 persons, 2 benchs, 268.3ms
Speed: 2.0ms preprocess, 268.3ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 3 persons, 2 benchs, 262.3ms
Speed: 4.0ms preprocess, 262.3ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 3 persons, 1 bench, 254.3ms
Speed: 4.0ms preprocess, 254.3ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 3 persons, 1 bench, 1 sports ball, 250.3ms
Speed: 3.0ms preprocess, 250.3ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 3 persons, 2 benchs, 250.3ms
Speed: 3.0ms preprocess, 250.3ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 3 persons, 1 


0: 320x640 3 persons, 251.2ms
Speed: 4.2ms preprocess, 251.2ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 3 persons, 259.0ms
Speed: 3.0ms preprocess, 259.0ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 3 persons, 250.3ms
Speed: 3.0ms preprocess, 250.3ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 3 persons, 264.3ms
Speed: 4.0ms preprocess, 264.3ms inference, 3.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 3 persons, 256.3ms
Speed: 4.0ms preprocess, 256.3ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 3 persons, 268.3ms
Speed: 4.0ms preprocess, 268.3ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 3 persons, 255.3ms
Speed: 3.0ms preprocess, 255.3ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 3 persons, 253.3ms
Speed: 3.0ms preprocess, 253.3ms inference, 2.0ms postprocess per 


0: 320x640 3 persons, 2 tvs, 288.2ms
Speed: 4.0ms preprocess, 288.2ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 2 persons, 2 tvs, 273.3ms
Speed: 4.0ms preprocess, 273.3ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 2 persons, 1 tv, 270.3ms
Speed: 5.0ms preprocess, 270.3ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 2 persons, 1 tv, 269.3ms
Speed: 5.0ms preprocess, 269.3ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 3 persons, 2 tvs, 278.3ms
Speed: 3.0ms preprocess, 278.3ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 2 persons, 1 tv, 281.4ms
Speed: 4.0ms preprocess, 281.4ms inference, 1.9ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 2 persons, 1 tv, 271.3ms
Speed: 4.0ms preprocess, 271.3ms inference, 3.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 2 persons, 2 tvs, 282.4ms
Speed: 5.0ms p

Speed: 3.0ms preprocess, 169.6ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 2 persons, 1 tv, 176.5ms
Speed: 2.0ms preprocess, 176.5ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 2 persons, 1 tv, 181.5ms
Speed: 3.0ms preprocess, 181.5ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person, 1 tv, 240.4ms
Speed: 3.0ms preprocess, 240.4ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person, 1 tv, 259.3ms
Speed: 4.0ms preprocess, 259.3ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person, 1 tv, 240.3ms
Speed: 4.0ms preprocess, 240.3ms inference, 3.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person, 1 tv, 246.3ms
Speed: 3.0ms preprocess, 246.3ms inference, 3.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person, 1 tv, 246.3ms
Speed: 3.0ms preprocess, 246.3ms inference, 2.0ms postproces


0: 320x640 2 persons, 1 bicycle, 266.3ms
Speed: 4.0ms preprocess, 266.3ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 2 persons, 1 bicycle, 251.4ms
Speed: 3.0ms preprocess, 251.4ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 2 persons, 1 bicycle, 173.5ms
Speed: 3.0ms preprocess, 173.5ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 2 persons, 1 bicycle, 1 tennis racket, 179.5ms
Speed: 2.0ms preprocess, 179.5ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person, 1 tennis racket, 177.5ms
Speed: 2.0ms preprocess, 177.5ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person, 1 tennis racket, 1 tv, 178.5ms
Speed: 3.0ms preprocess, 178.5ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person, 1 bicycle, 1 tennis racket, 1 tv, 178.5ms
Speed: 2.0ms preprocess, 178.5ms inference, 1.0ms postprocess p